### So here i used MovingAverage Crossover Strategy to predict a Entry price and Exit price 
### I have used the following strategy because it's simple to use and to implement and mostly it's used widely by the traders into trading using several confluences . Also i haven't solely used a Youtube video but videos to make this algorithm.also I used several websites to get to know more about the strategy.

## <b> So what is Moving Average Crossover Strategy? </b>
### Basically we use moving average indicators with a window of 20 days(Short window or short_moving Average) and window of 50 days(Long Moving Average) and whenever the ShortMA crosses the LongMA from below we place ourselves for a Entry in a trade and vice versa when the ShortMA crosses the LongMA from the above we position ourselves for a selling position. so here we have used the same strategy but with a limited profit and stop loss which is in the ratio of 3:1 which means that our profit will be 3% of the entry price and if something goes wrong in the trade and the price goes down instead of going up after buying the system will automatically exit the trade with exit price as 1% of the entry price.
 

In [3]:
#Importing all the neccessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import yfinance as yf
from datetime import date

In [4]:
#Importing Stock data
HUL = yf.download('HINDUNILVR.NS',start=date(2018,1,1),end=date(2022,1,1))

[*********************100%***********************]  1 of 1 completed


1 Failed download:
['HINDUNILVR.NS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2018-01-01 -> 2022-01-01)')


In [5]:
HUL.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [6]:
#Function to get the entry and exit price
moving_average_crossover_strategy(HUL, short_window=20, long_window=50)


In [7]:
def moving_average_crossover_strategy(data, short_window, long_window):
    # Compute the short-term and long-term moving averages
    data['short_moving_average'] = data['Close'].rolling(window=short_window).mean()  #Short Moving average with 20 days
    data['long_moving_average'] = data['Close'].rolling(window=long_window).mean()    #Long term moving average with 50 days

    # Initialize variables
    is_in_trade = False            #A flag to Check if we're in trade or not     
    entry_price = 0.0              # Initlizing the value of entry price
    stop_loss = 0.01  # 1% stop loss
    take_profit = 0.03  # 3% take profit


    # Create a candlestick trace
    candlestick_trace = go.Candlestick(
        x=data.index,                 
        open=data['Open'],
        high=data['High'],
        low=data['Low'],
        close=data['Close'],
        name='Candlestick'
    )

    # Create a trace for the short-term moving average
    short_ma_trace = go.Scatter(
        x=data.index,
        y=data['short_moving_average'],
        mode='lines',
        name=f'Short MA ({short_window})'
    )

    # Create a trace for the long-term moving average
    long_ma_trace = go.Scatter(
        x=data.index,
        y=data['long_moving_average'],
        mode='lines',
        name=f'Long MA ({long_window})'
    )

    # Create the figure and add the traces
    fig = go.Figure(data=[candlestick_trace, short_ma_trace, long_ma_trace])
    fig.update_layout(xaxis_rangeslider_visible=False)

    # Iterate over the data
    for i in range(1, len(data)):
        if not is_in_trade:
            # Check for a crossover event
            if data['short_moving_average'][i] > data['long_moving_average'][i] and data['short_moving_average'][i - 1] <= data['long_moving_average'][i - 1]:
                # Enter the trade
                is_in_trade = True
                entry_price = data['Close'][i]
                print('Enter trade at price:', entry_price)
        else:
            # Check for an exit condition
            stop_loss_price = entry_price * (1 - stop_loss)
            take_profit_price = entry_price * (1 + take_profit)

            if data['Close'][i] <= stop_loss_price:
                # Exit the trade due to stop loss hit
                is_in_trade = False
                exit_price = data['Close'][i]
                print('Exit trade at price:', exit_price)
            elif data['Close'][i] >= take_profit_price:
                # Exit the trade due to take profit hit
                is_in_trade = False
                exit_price = data['Close'][i]
                print('Exit trade at price:', exit_price)
            elif data['short_moving_average'][i] < data['long_moving_average'][i] and data['short_moving_average'][i - 1] >= data['long_moving_average'][i - 1]:
                # Exit the trade due to crossover event in the opposite direction
                is_in_trade = False
                exit_price = data['Close'][i]
                print('Exit trade at price:', exit_price)

    # If the trade is still open at the end of the data, close it
    if is_in_trade:
        is_in_trade = False
        exit_price = data['Close'].iloc[-1]
        print('Exit trade at price:', exit_price)

    # Show the figure
    fig.show()